# Tutorial Interativo - Fase 1: LangGraph e Estado Central

Este tutorial ensina como usar o sistema de estado central baseado em LangGraph do Gianna.

## Objetivos
1. Compreender o sistema de estado do Gianna
2. Usar LangGraphChain para processamento stateful
3. Gerenciar sessões e persistência
4. Integrar com o sistema existente

## Pré-requisitos
- Gianna instalado (`poetry install`)
- Chave OpenAI configurada no `.env`
- Conhecimento básico de Python

In [ ]:
# Importações necessárias
import os
import time
from datetime import datetime
from pathlib import Path

# Importações do Gianna
from gianna.core.langgraph_chain import LangGraphChain
from gianna.core.state_manager import StateManager
from gianna.core.state import GiannaState, ConversationState, AudioState, CommandState
from gianna.assistants.models.factory_method import get_chain_instance

print("✅ Importações concluídas")
print(f"📍 Diretório atual: {os.getcwd()}")

## 1. Configuração Inicial

Primeiro, vamos configurar o ambiente e verificar se tudo está funcionando.

In [ ]:
# Verificar configuração
from dotenv import load_dotenv
load_dotenv()

# Verificar se chave OpenAI está configurada
openai_key = os.getenv("OPENAI_API_KEY")
if openai_key:
    print(f"✅ OpenAI API Key configurada (primeiros 10 chars: {openai_key[:10]}...)")
else:
    print("❌ OpenAI API Key não encontrada. Configure no arquivo .env")
    print("Exemplo: OPENAI_API_KEY=sk-...")

# Configurações do tutorial
TUTORIAL_SESSION_ID = f"tutorial_session_{int(time.time())}"
DATABASE_PATH = "tutorial_gianna_state.db"

print(f"🆔 Session ID: {TUTORIAL_SESSION_ID}")
print(f"💾 Database: {DATABASE_PATH}")

## 2. Criando um StateManager

O StateManager é o coração do sistema de estado do Gianna.

In [ ]:
# Criar StateManager
state_manager = StateManager(db_path=DATABASE_PATH)

print("🧠 StateManager criado!")
print(f"📄 Base de dados: {state_manager.db_path}")
print(f"🔧 Checkpointer: {type(state_manager.checkpointer).__name__}")

# Verificar se base de dados foi criada
if Path(DATABASE_PATH).exists():
    size = Path(DATABASE_PATH).stat().st_size
    print(f"💽 Tamanho da base de dados: {size} bytes")
else:
    print("⚠️ Base de dados ainda não existe (será criada na primeira operação)")

## 3. Criando uma LangGraphChain

A LangGraphChain substitui as chains tradicionais com capacidades de estado.

In [ ]:
# Criar uma LangGraphChain
assistant_chain = LangGraphChain(
    model_name="gpt35",  # Ou "gpt4" se preferir
    prompt="""Você é um assistente inteligente e amigável chamado Gianna.
    
Características:
- Seja conversacional e natural
- Lembre-se do contexto das conversas anteriores
- Seja útil e informativo
- Responda em português
- Se apresente como Gianna quando apropriado

Você tem acesso ao histórico da conversação e pode referenciar
mensagens anteriores para fornecer respostas mais contextuais.""",
    temperature=0.7,
    max_tokens=500
)

print("🤖 LangGraphChain criada!")
print(f"🧠 Modelo: {assistant_chain.model_name}")
print(f"📊 Workflow compilado: {assistant_chain.graph is not None}")

# Verificar estrutura do workflow
if hasattr(assistant_chain, 'graph') and assistant_chain.graph:
    print("\n🔄 Estrutura do Workflow:")
    # Listar nós do workflow
    if hasattr(assistant_chain.graph, 'nodes'):
        print(f"   📋 Nós: {list(assistant_chain.graph.nodes.keys())}")

## 4. Primeira Interação com Estado

Vamos fazer nossa primeira interação e ver como o estado é gerenciado.

In [ ]:
# Primeira interação
print("🎯 Primeira interação com o assistente...")

response1 = assistant_chain.invoke({
    "input": "Olá! Meu nome é João e eu gostaria de conhecer o Gianna."
}, session_id=TUTORIAL_SESSION_ID)

print("\n👤 Usuário: Olá! Meu nome é João e eu gostaria de conhecer o Gianna.")
print(f"🤖 Gianna: {response1['output']}")

# Verificar metadados da resposta
if 'metadata' in response1:
    print("\n📊 Metadados da resposta:")
    for key, value in response1['metadata'].items():
        if key != 'workflow_steps':  # Não mostrar steps por serem muito verbosos
            print(f"   • {key}: {value}")

## 5. Continuando a Conversa - Testando Memória

Vamos ver se o assistente se lembra do contexto anterior.

In [ ]:
# Segunda interação - testando memória
print("🧠 Testando memória de contexto...")

response2 = assistant_chain.invoke({
    "input": "Você se lembra do meu nome? E pode me contar mais sobre suas capacidades?"
}, session_id=TUTORIAL_SESSION_ID)

print("\n👤 Usuário: Você se lembra do meu nome? E pode me contar mais sobre suas capacidades?")
print(f"🤖 Gianna: {response2['output']}")

# Verificar se o nome foi lembrado
if "João" in response2['output'] or "joão" in response2['output']:
    print("\n✅ Memória funcionando! O assistente lembrou do nome.")
else:
    print("\n⚠️ Possível problema de memória - nome não foi mencionado.")

## 6. Inspecionando o Estado da Sessão

Vamos examinar como o estado está sendo armazenado.

In [ ]:
# Obter estado atual da sessão
try:
    current_state = assistant_chain.get_current_state(TUTORIAL_SESSION_ID)
    
    print("📊 Estado atual da sessão:")
    
    # Informações da conversação
    if 'conversation' in current_state:
        conv_state = current_state['conversation']
        print(f"\n💬 Conversação:")
        print(f"   • Mensagens: {len(conv_state.get('messages', []))}")
        print(f"   • Session ID: {conv_state.get('session_id', 'N/A')}")
        
        # Mostrar últimas mensagens
        messages = conv_state.get('messages', [])
        if messages:
            print(f"\n📝 Histórico da conversa:")
            for i, msg in enumerate(messages[-4:], 1):  # Últimas 4 mensagens
                role = "👤" if msg.get('role') == 'user' else "🤖"
                content = msg.get('content', '')[:100] + "..." if len(msg.get('content', '')) > 100 else msg.get('content', '')
                timestamp = msg.get('timestamp', 'N/A')
                print(f"   {i}. {role} [{timestamp}]: {content}")
    
    # Informações de áudio
    if 'audio' in current_state:
        audio_state = current_state['audio']
        print(f"\n🔊 Estado de Áudio:")
        print(f"   • Modo atual: {audio_state.get('current_mode', 'N/A')}")
        print(f"   • Idioma: {audio_state.get('language', 'N/A')}")
    
    # Metadados
    if 'metadata' in current_state:
        print(f"\n🏷️ Metadados:")
        metadata = current_state['metadata']
        for key, value in list(metadata.items())[:5]:  # Primeiros 5
            print(f"   • {key}: {value}")
            
except Exception as e:
    print(f"⚠️ Erro ao obter estado: {str(e)}")
    print("Isso pode ser normal em implementações simplificadas.")

## 7. Trabalhando com Múltiplas Sessões

Vamos criar uma segunda sessão para testar isolamento de estado.

In [ ]:
# Criar segunda sessão
SECOND_SESSION_ID = f"tutorial_session_2_{int(time.time())}"

print(f"🆔 Nova sessão criada: {SECOND_SESSION_ID}")

# Interagir com a segunda sessão
response3 = assistant_chain.invoke({
    "input": "Oi! Eu sou Maria. Esta é nossa primeira conversa?"
}, session_id=SECOND_SESSION_ID)

print("\n🎭 === SEGUNDA SESSÃO ===")
print("👤 Maria: Oi! Eu sou Maria. Esta é nossa primeira conversa?")
print(f"🤖 Gianna: {response3['output']}")

# Verificar se o assistente não confunde as sessões
if "João" in response3['output'] or "joão" in response3['output']:
    print("\n⚠️ Possível vazamento de estado entre sessões!")
else:
    print("\n✅ Isolamento de sessões funcionando corretamente!")

# Voltar para primeira sessão
response4 = assistant_chain.invoke({
    "input": "Você ainda se lembra de mim?"
}, session_id=TUTORIAL_SESSION_ID)

print("\n🎭 === VOLTANDO À PRIMEIRA SESSÃO ===")
print("👤 João: Você ainda se lembra de mim?")
print(f"🤖 Gianna: {response4['output']}")

## 8. Integração com Factory Method

Vamos testar a compatibilidade com o sistema de factory existente.

In [ ]:
# Usar factory method tradicional
print("🏭 Testando integração com Factory Method...")

# Isso deve retornar uma LangGraphChain automaticamente
factory_chain = get_chain_instance(
    "gpt35",
    "Você é um assistente técnico especializado em programação."
)

print(f"🔧 Tipo da chain: {type(factory_chain).__name__}")
print(f"🔄 É LangGraphChain? {isinstance(factory_chain, LangGraphChain)}")

# Testar compatibilidade de interface
test_response = factory_chain.invoke({
    "input": "Como funciona o padrão Factory Method em Python?"
})

print("\n👤 Usuário: Como funciona o padrão Factory Method em Python?")
print(f"🤖 Assistente: {test_response['output'][:200]}...")

# Verificar se resposta tem formato esperado
if isinstance(test_response, dict) and 'output' in test_response:
    print("\n✅ Interface compatível com sistema existente!")
else:
    print("\n❌ Problema de compatibilidade de interface.")

## 9. Persistência e Recuperação de Estado

Vamos testar se o estado persiste entre execuções.

In [ ]:
# Verificar persistência
print("💾 Testando persistência de estado...")

# Salvar informação importante
save_response = assistant_chain.invoke({
    "input": "Lembre-se: meu número da sorte é 42 e minha cor favorita é azul."
}, session_id=TUTORIAL_SESSION_ID)

print("👤 João: Lembre-se: meu número da sorte é 42 e minha cor favorita é azul.")
print(f"🤖 Gianna: {save_response['output']}")

# Simular "reinicialização" criando nova instância
print("\n🔄 Simulando reinicialização do sistema...")

# Nova instância da chain (usando mesmo estado)
new_chain = LangGraphChain(
    model_name="gpt35",
    prompt="""Você é um assistente inteligente e amigável chamado Gianna.
    Lembre-se sempre do contexto das conversas anteriores."""
)

# Testar se estado foi recuperado
recovery_response = new_chain.invoke({
    "input": "Qual é o meu número da sorte e minha cor favorita?"
}, session_id=TUTORIAL_SESSION_ID)

print("\n👤 João: Qual é o meu número da sorte e minha cor favorita?")
print(f"🤖 Gianna: {recovery_response['output']}")

# Verificar se informações foram recuperadas
response_text = recovery_response['output'].lower()
if "42" in response_text and "azul" in response_text:
    print("\n✅ Estado persistido e recuperado com sucesso!")
elif "42" in response_text or "azul" in response_text:
    print("\n⚠️ Estado parcialmente recuperado.")
else:
    print("\n❌ Estado não foi persistido adequadamente.")

## 10. Monitoramento e Métricas

Vamos verificar métricas de performance e uso.

In [ ]:
# Coletar métricas
print("📊 Coletando métricas do sistema...")

# Verificar tamanho da base de dados
if Path(DATABASE_PATH).exists():
    db_size = Path(DATABASE_PATH).stat().st_size
    print(f"💽 Tamanho da base de dados: {db_size:,} bytes")
    
    # Calcular número de interações
    interactions_count = 0
    try:
        current_state = assistant_chain.get_current_state(TUTORIAL_SESSION_ID)
        if 'conversation' in current_state and 'messages' in current_state['conversation']:
            interactions_count = len(current_state['conversation']['messages'])
    except:
        interactions_count = "N/A"
    
    print(f"💬 Interações na sessão principal: {interactions_count}")

# Teste de performance
print("\n⚡ Teste de performance...")

start_time = time.time()
perf_response = assistant_chain.invoke({
    "input": "Esta é apenas uma mensagem de teste rápida."
}, session_id=TUTORIAL_SESSION_ID)
end_time = time.time()

response_time = end_time - start_time
print(f"⏱️ Tempo de resposta: {response_time:.2f} segundos")

if response_time < 2.0:
    print("🟢 Performance excelente!")
elif response_time < 5.0:
    print("🟡 Performance aceitável.")
else:
    print("🔴 Performance pode ser melhorada.")

# Informações do sistema
import psutil
print(f"\n🖥️ Uso de memória: {psutil.virtual_memory().percent:.1f}%")
print(f"💾 Uso de disco: {psutil.disk_usage('.').percent:.1f}%")

## 11. Limpeza e Finalização

Vamos limpar os recursos e finalizar o tutorial.

In [ ]:
# Mensagem final
final_response = assistant_chain.invoke({
    "input": "Obrigado por este tutorial! Foi muito útil aprender sobre LangGraph."
}, session_id=TUTORIAL_SESSION_ID)

print("👤 João: Obrigado por este tutorial! Foi muito útil aprender sobre LangGraph.")
print(f"🤖 Gianna: {final_response['output']}")

# Estatísticas finais
print("\n" + "="*60)
print("📊 RESUMO DO TUTORIAL")
print("="*60)

try:
    final_state = assistant_chain.get_current_state(TUTORIAL_SESSION_ID)
    if 'conversation' in final_state and 'messages' in final_state['conversation']:
        total_messages = len(final_state['conversation']['messages'])
        user_messages = sum(1 for msg in final_state['conversation']['messages'] if msg.get('role') == 'user')
        assistant_messages = sum(1 for msg in final_state['conversation']['messages'] if msg.get('role') == 'assistant')
        
        print(f"💬 Total de mensagens: {total_messages}")
        print(f"👤 Mensagens do usuário: {user_messages}")
        print(f"🤖 Mensagens do assistente: {assistant_messages}")
except:
    print("📊 Estatísticas de conversação não disponíveis")

print(f"🆔 Sessões utilizadas: 2")
print(f"💾 Base de dados: {DATABASE_PATH}")
print(f"⏱️ Tutorial iniciado às: {datetime.now().strftime('%H:%M:%S')}")

print("\n✅ Tutorial concluído com sucesso!")
print("\n💡 Principais conceitos aprendidos:")
print("   • Como criar e usar LangGraphChain")
print("   • Gerenciamento de estado e sessões")
print("   • Persistência automática de conversações")
print("   • Integração com factory methods existentes")
print("   • Isolamento entre sessões diferentes")
print("   • Monitoramento de performance")

print("\n🚀 Próximos passos sugeridos:")
print("   • Explorar o tutorial da Fase 2 (Multi-Agentes)")
print("   • Experimentar com diferentes modelos LLM")
print("   • Integrar com sistema de áudio (Fase 3)")
print("   • Desenvolver aplicações customizadas")

## Próximos Passos

Agora que você domina os conceitos básicos da Fase 1, pode:

1. **Experimentar diferentes prompts**: Modifique o prompt do sistema para criar assistentes especializados
2. **Explorar outros modelos**: Teste com "gpt4", "claude", "gemini", etc.
3. **Integrar com aplicações existentes**: Use o padrão aprendido em seus próprios projetos
4. **Avançar para Fase 2**: Aprenda sobre sistema multi-agente no próximo tutorial

### Recursos Adicionais

- [Documentação da API Core](../../docs/api/core/)
- [Guia do Desenvolvedor](../../docs/developer-guide/)
- [Exemplos Avançados](../../examples/advanced/)

### Suporte

Se encontrou problemas ou tem dúvidas:
1. Verifique a [documentação completa](../../docs/)
2. Consulte os [exemplos práticos](../../examples/)
3. Revise as configurações do `.env`